<a href="https://colab.research.google.com/github/Mfys212/Generasi-Information-Teks-from-Kompas/blob/main/Generasi_Teks_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pyspark
!pip install pytorch-accelerated
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=559b378598cf03d361e1a0edf77af949101b9394f946ddfd53d34c0f424f405f
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 4.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.wh

In [15]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, size, col, to_timestamp
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import requests
import os

In [4]:
def download_file(url, file_name):
  response = requests.get(url)
  with open(file_name, 'wb') as f:
    f.write(response.content)

github_url = "https://github.com/Mfys212/Generasi-Information-Teks-from-Kompas/raw/main/"
file_names = ["ai.csv", "algoritma.csv", "biologi.csv", "data (3).csv",
              "fisika.csv", "kimia.csv", "komputer.csv", "matematika.csv",
              "pemrograman.csv", "sains.csv", "statistika.csv",
              "teknik.csv", "teknologi.csv"]

if not os.path.exists("downloaded_files"):
  os.makedirs("downloaded_files")

for file_name in file_names:
  file_url = github_url + file_name
  file_path = os.path.join("downloaded_files", file_name)
  print(f"downloading {file_name} ....")
  download_file(file_url, file_path)

downloading ai.csv ....
downloading algoritma.csv ....
downloading biologi.csv ....
downloading data (3).csv ....
downloading fisika.csv ....
downloading kimia.csv ....
downloading komputer.csv ....
downloading matematika.csv ....
downloading pemrograman.csv ....
downloading sains.csv ....
downloading statistika.csv ....
downloading teknik.csv ....
downloading teknologi.csv ....


In [12]:
df = pd.read_csv("/content/downloaded_files/" + file_names[0])

for file_name in file_names[1:]:
  df_ = pd.read_csv("/content/downloaded_files/" + file_name)
  df = pd.concat([df, df_], ignore_index=True)

spark = SparkSession.builder \
    .appName("Kompas Data") \
    .getOrCreate()

df = spark.createDataFrame(df)

df.printSchema()
df.show()

root
 |-- Tanggal: string (nullable = true)
 |-- Judul: string (nullable = true)
 |-- Konten: string (nullable = true)
 |-- Link: string (nullable = true)

+-------------------+--------------------+--------------------+--------------------+
|            Tanggal|               Judul|              Konten|                Link|
+-------------------+--------------------+--------------------+--------------------+
|2024-05-09 14:30:00|Kontroversi Reka ...|LONDON, KOMPAS.co...|http://www.kompas...|
|2024-05-06 10:31:00|X Twitter Rilis F...|KOMPAS.com- Media...|http://tekno.komp...|
|2024-05-06 15:01:00|Mengenal Gemini A...|KOMPAS.com- Artif...|http://tekno.komp...|
|2024-05-11 10:05:00|Siapkan Generasi ...|KOMPAS.com -Sekol...|http://www.kompas...|
|2024-05-06 12:41:00|[HOAKS] Foto Rest...|hoaks!Berdasarkan...|http://www.kompas...|
|2024-05-10 15:35:00|[HOAKS] Foto Riha...|hoaks!Berdasarkan...|http://www.kompas...|
|2024-05-07 17:58:00|Hoaks, Spongebob ...|KOMPAS.com- Sebua...|http://www.kompa

In [11]:
num_rows = df.count()
all_content = "\n".join(df.select("Konten").rdd.map(lambda row: row[0]).collect())

with open("file.txt", "w") as file:
  file.write(all_content)

with open("file.txt", "r") as file:
  content = file.read()

words = content.split()
num_words = len(words)

df = df.withColumn("Tanggal", to_timestamp("Tanggal", "yyyy-MM-dd HH:mm:ss"))
min_date = df.select("tanggal").agg({"tanggal": "min"}).collect()[0][0]
max_date = df.select("tanggal").agg({"tanggal": "max"}).collect()[0][0]

print("Jumlah berita:", num_rows)
print("Jumlah kata dalam berita:", num_words)
print("Berita paling lama:", min_date)
print("Berita paling baru:", max_date)

Jumlah berita: 28839
Jumlah kata dalam berita: 8913668
Berita paling lama: 2013-05-03 15:58:00
Berita paling baru: 2024-05-12 21:00:00


In [14]:
with open("file.txt", "r") as file:
  content = file.read()

wordcloud = WordCloud(width=800, height=400, background_color='white').generate(content)

plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud')
plt.show()

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('indonesian-nlp/gpt2-medium-indonesian')
model = GPT2LMHeadModel.from_pretrained('indonesian-nlp/gpt2-medium-indonesian')

# Load your training data and tokenize it
train_data_file = "path/to/your/training/data.txt"
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=train_data_file,
    block_size=128  # Adjust according to your needs
)

# Initialize data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Training arguments
training_args = TrainingArguments(
    output_dir="./fine-tuned-gpt2-indonesian",
    overwrite_output_dir=True,
    num_train_epochs=3,  # Adjust as needed
    per_device_train_batch_size=4,  # Adjust as needed
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# Start training
trainer.train()

# Save the model
trainer.save_model("./fine-tuned-gpt2-indonesian")


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


AttributeError: 

In [ ]:
len(output)

2

In [ ]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='indonesian-nlp/gpt2-medium-indonesian')
set_seed(42)
generator("Hai saya orang indonesia,", max_length=30, num_return_sequences=5)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Hai saya orang indonesia, mau tanya,apa betul produk tersebut adalah produk indonesia,saya ingin beli produk tersebut,saya tinggal di jakarta, dan saya baru'},
 {'generated_text': 'Hai saya orang indonesia, saat ini saya baru mulai mencari jalan keluar dari sebuah kemiskinan. bagaimana saya mau kaya kalau hidup miskin, apakah saya bisa mencari sesuatu'},
 {'generated_text': 'Hai saya orang indonesia,saya ingin bertanya,jika di jawa tengah,persib bandung atau persib vs arema ada berita baru dan siaran langsung nya'},
 {'generated_text': 'Hai saya orang indonesia, ingin bekerja untuk negeri ini untuk membangun negara yang kita cintai. Saya senang menulis, jadi ketika saya ada waktu luang saya akan menulis'},
 {'generated_text': 'Hai saya orang indonesia,saya berumur 16th, saya tinggal dibogor.apakah bisa saya bergabung menjadi anggota komunitas ini?\nBisa saja, tapi untuk'}]